In [1]:
import os
import polars as pl
import matplotlib.pyplot as plt

## Data reading

- Load all 2024 months of taxi rides  

- Also load taxi zone lookup data  

- Include only rides starting in 2024 and ending at most at 01.01.2025  

- Optimize data types, particularly for integers and categorical strings  


In [2]:
DATA_DIR = "data"  

dfs = []
for month in range(1, 13):
    df = (
        pl.scan_parquet(f"{DATA_DIR}/{month}.parquet")
        .with_columns(
            pl.col("tpep_pickup_datetime").dt.cast_time_unit("ms"),
            pl.col("tpep_dropoff_datetime").dt.cast_time_unit("ms"),
        )
        .filter(
            pl.col("tpep_pickup_datetime").dt.year() == 2024,
            pl.col("tpep_dropoff_datetime") < pl.datetime(2025, 1, 1, time_unit="ms"),
        )
    )
    dfs.append(df)

df_2024 = pl.concat(dfs)

In [3]:



zones = pl.scan_csv("data/taxi_zone_lookup.csv")



In [4]:
df_2024 = df_2024.with_columns(
    pl.col("VendorID").cast(pl.Int8),
    pl.col("passenger_count").cast(pl.Int8),
    pl.col("RatecodeID").cast(pl.Int8),
    pl.col("PULocationID").cast(pl.Int16),
    pl.col("DOLocationID").cast(pl.Int16),
    pl.col("payment_type").cast(pl.Int8),
    pl.col("store_and_fwd_flag").cast(pl.Categorical),
    pl.col("fare_amount").cast(pl.Float32),
    pl.col("extra").cast(pl.Float32),
    pl.col("mta_tax").cast(pl.Float32),
    pl.col("tip_amount").cast(pl.Float32),
    pl.col("tolls_amount").cast(pl.Float32),
    pl.col("improvement_surcharge").cast(pl.Float32),
    pl.col("total_amount").cast(pl.Float32),
    pl.col("congestion_surcharge").cast(pl.Float32),
    pl.col("Airport_fee").cast(pl.Float32),
    pl.col("trip_distance").cast(pl.Float32),
)

## Data cleaning and filtering 


- Fill NULL values in `passengers_count` with 1
- Remove rides with zero passengers
- If a ride has over 6 passengers, replace the value with 6
- Remove rides lasting over 2 hours
- In all money-related columns, replace values with absolute value to fix negative amounts
- Remove rides with over 1000 dollars in any money-related column
- Remove rows with values of `RatecodeID` or `VendorID` missing or outside those defined in the data dictionary


In [5]:
MN_COLS = [
    "fare_amount",
    "extra",
    "mta_tax",
    "tip_amount",
    "tolls_amount",
    "improvement_surcharge",
    "congestion_surcharge",
    "Airport_fee",
    "total_amount",
]

POSSIBLE_VENDOR = [1, 2]
POSSIBLE_RATE   = [1, 2, 3, 4, 5, 6]

df_clean = (
    df_2024
    .with_columns( 
        pl.col("passenger_count").fill_null(1).alias("passenger_count")
    )
    .filter(
        pl.col("passenger_count") != 0
    )

    .with_columns( 
        pl.when(pl.col("passenger_count") > 6).then(6).otherwise(pl.col("passenger_count"))
        .alias("passenger_count")
    )

    .with_columns(
        ((pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime")) / pl.duration(minutes=1))
        .alias("trip_minutes")
    )

    .filter(
        pl.col("trip_minutes").is_not_null() &
        (pl.col("trip_minutes") > 0) &
        (pl.col("trip_minutes") <= 120)
    )

    .with_columns(
        [pl.col(c).abs().alias(c) for c in MN_COLS]
    )

    .with_columns(
        pl.max_horizontal(
            [pl.col(c).fill_null(0) for c in MN_COLS]).alias("max_money")
        )

    .filter(pl.col("max_money") <= 1000).drop("max_money")

    .filter(
        pl.col("VendorID").is_in(POSSIBLE_VENDOR) & pl.col("VendorID").is_not_null() &
        pl.col("RatecodeID").is_in(POSSIBLE_RATE) & pl.col("RatecodeID").is_not_null()
    )

    .drop("trip_minutes")
)

## 3. Data transformation

- Combine payment type values for `"No charge"`, `"Dispute"`, `"Unknown"` and `"Voided trip"` into one type, so we have only credit card, cash, or other
- Replace `PULocationID` and `DOLocationID` with borough names by joining taxi zone lookup and removing unnecessary columns
- Add boolean variable `is_airport_ride`, true if there is non-zero airport fee
- Add boolean variable `is_rush_hour`, rush hours are defined as 6:30–9:30 and 15:30–20:00 during weekdays


In [ ]:
zones_lf = (
    pl.scan_csv("data/taxi_zone_lookup.csv")
      .select(["LocationID", "Borough"])
)

ts_morning  = pl.time(6,30)
te_morning    = pl.time(9,30)
ts_evening  = pl.time(15, 30)
te_evening    = pl.time(20, 0)

df_transformed = (
    df_clean 
    .with_columns(
        pl.when(pl.col("payment_type") == 1).then(pl.lit("credit_card"))
         .when(pl.col("payment_type") == 2).then(pl.lit("cash"))
         .otherwise(pl.lit("other"))
         .alias("payment_type")
    )
    .join(
        zones_lf.rename({"Borough": "pu_borough"}),
        left_on="PULocationID", right_on="LocationID"
    )
    .join(
        zones_lf.rename({"Borough": "do_borough"}),
        left_on="DOLocationID", right_on="LocationID"
    )
    .with_columns(
        (pl.col("Airport_fee").fill_null(0) > 0).alias("is_airport_ride")
    )
    .with_columns(
        [
            pl.col("tpep_pickup_datetime").dt.weekday().alias("weekday"),
            pl.col("tpep_pickup_datetime").dt.time().alias("pickup_time"),
        ]
    )
    .with_columns(
        (
            (pl.col("weekday") <= 4) & (
                ((pl.col("pickup_time") >= ts_morning) & (pl.col("pickup_time") < te_morning)) |
                ((pl.col("pickup_time") >= ts_evening) & (pl.col("pickup_time") < te_evening))
            )
        ).alias("is_rush_hour")
    )
    .drop(["LocationID", "weekday", "pickup_time"])  
    .drop(["PULocationID", "DOLocationID"])
)

## 4. Feature extraction

- Apply dummy encoding to features:
  - payment type
  - pickup borough
  - dropoff borough

- Add integer variables for counting daily events:
  - total number of rides (target variable)
  - number of airport rides
  - number of rush hour rides

- Add features aggregating daily rides information:
  - average fare amount
  - median distance
  - sum of total amounts
  - total amount paid by card, cash, and other
  - total congestion surcharge
  - total number of passengers

- Add time features:
  - `quarter`
  - `month`
  - `day_of_month`
  - `day_of_week`
  - `is_weekend` (boolean)

- Add column `date`, indicating day with given features
- Make sure to exclude other columns unnecessary for machine learning (IDs, timestamps, unused financial information)
- Properly name all columns
- Save results as `dataset.parquet` file

## 5. Data analysis

- Print shape
- Show top rows
- Describe statistics
- Print schema

- Plot target variable (daily number of rides):
  - Histogram of values
  - Line plot: date vs number of rides (remember to sort by date)